In [27]:
import warnings
import pandas as pd
import torch
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_forecasting import TimeSeriesDataSet, GroupNormalizer
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.models.deepar import DeepAR
from pytorch_forecasting.metrics import NormalDistributionLoss

warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)

def main():
    pl.seed_everything(42, workers=True)

    # 1. 加载和预处理
    data = pd.read_csv("/kaggle/input/aaplll/stock_data.csv")
    data["Date"] = pd.to_datetime(data["Date"], format="%Y/%m/%d")
    data = data.sort_values("Date").reset_index(drop=True)
    data["time_idx"] = data.index + 1
    data["static"] = data["stock_id"]
    
    # 提取更多时间特征
    data["day_of_week"] = data["Date"].dt.dayofweek
    data["day_of_week"] = data["day_of_week"].astype(str)  # 或者：.astype("category")
    data["day_of_month"] = data["Date"].dt.day
    data["week_of_year"] = data["Date"].dt.isocalendar().week.astype(int)

    max_encoder_length = 50
    max_prediction_length = 10
    training_cutoff = data["time_idx"].max() - (max_encoder_length + max_prediction_length)
    
    # 2. 定义训练集
    training = TimeSeriesDataSet(
        data=data[data.time_idx <= training_cutoff],
        time_idx="time_idx",
        target="Close",
        group_ids=["stock_id"],
        categorical_encoders={
            "stock_id": NaNLabelEncoder().fit(data["stock_id"])
        },
        static_categoricals=["static"],
        time_varying_known_categoricals=["day_of_week"],  # 示例
        time_varying_known_reals=["time_idx", "day_of_month", "week_of_year"],  # 示例
        time_varying_unknown_reals=["Close"],
        min_encoder_length=max_encoder_length,
        max_encoder_length=max_encoder_length,
        min_prediction_length=max_prediction_length,
        max_prediction_length=max_prediction_length,
        target_normalizer=GroupNormalizer(groups=["stock_id"], transformation="log"),
        add_relative_time_idx=True,
        add_target_scales=True,
        randomize_length=True,  # 或者 False，看实际情况
    )

    # 3. 定义验证集
    validation_data = data[data.time_idx > training_cutoff]
    validation = TimeSeriesDataSet.from_dataset(
        training,
        validation_data,
        stop_randomization=True,
    )

    train_dataloader = training.to_dataloader(
        train=True, batch_size=128, num_workers=20, persistent_workers=True
    )
    val_dataloader = validation.to_dataloader(
        train=False, batch_size=128, num_workers=20, persistent_workers=True
    )

    # 回调
    early_stop_callback = EarlyStopping(
        monitor="val_loss", min_delta=1e-4, patience=100, verbose=False, mode="min"
    )
    lr_logger = LearningRateMonitor()
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath="checkpoints/",
        filename="deepar-{epoch:02d}-{val_loss:.2f}",
        save_top_k=3,
        mode="min",
    )

    # 4. Trainer
    trainer = pl.Trainer(
        max_epochs=100,
        accelerator="gpu", 
        devices="auto",
        precision="16-mixed",  # 尝试混合精度
        gradient_clip_val=0.1,
        callbacks=[lr_logger, early_stop_callback, checkpoint_callback],
    )

    # 5. DeepAR 模型
    deepar = DeepAR.from_dataset(
        training,
        learning_rate=0.01,   # 调低初始学习率
        hidden_size=128,      # 增大网络容量
        dropout=0.1,
        loss=NormalDistributionLoss(),
        log_interval=10,
        log_val_interval=5,
    )

    print(f"Number of parameters in network: {deepar.size() / 1e3:.1f}k")

    # 6. 开始训练
    trainer.fit(
        deepar,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    # 7. 评估
    actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
    predictions = deepar.predict(val_dataloader)
    device = predictions.device
    actuals = actuals.to(device)
    mae = (actuals - predictions).abs().mean()
    print(f"Mean absolute error of model: {mae.item():.4f}")

if __name__ == '__main__':
    main()

INFO: Seed set to 42
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:2

Number of parameters in network: 205.1k


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (19) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Mean absolute error of model: 18.0527


In [2]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing

In [4]:
!pip install pytorch-forecasting

In [22]:
import warnings
import pandas as pd
import torch
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, GroupNormalizer
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer
from pytorch_forecasting.metrics import MAE

# 忽略 pandas 的 SettingWithCopyWarning
warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)

def main():
    # 设置随机种子，确保结果可复现
    pl.seed_everything(42, workers=True)

    # 1. 加载数据
    data = pd.read_csv("/kaggle/input/vhzmdd/AAPL_stock_data_with_indicators.csv")

    # 2. 数据预处理
    data["Date"] = pd.to_datetime(data["Date"], format="%Y/%m/%d")
    data = data.sort_values("Date").reset_index(drop=True)
    data["time_idx"] = data.index + 1  # 时间索引从1开始
    data["stock_id"] = "AAPL"
    data["static"] = data["stock_id"]

    drop_cols = [col for col in data.columns if "#NAME?" in col]
    if drop_cols:
        print(f"剔除异常列: {drop_cols}")
        data = data.drop(columns=drop_cols)

    data = data.fillna(method='ffill')

    # 3. 定义 encoder 和 decoder 的步长
    max_encoder_length = 30
    max_prediction_length = 1

    training_cutoff = data["time_idx"].max() - (max_encoder_length + max_prediction_length)
    print(f"数据总长度: {data['time_idx'].max()}, 训练截止点: {training_cutoff}")

    # 4. 定义额外的技术指标
    additional_features = [
        "Volume", "SMA_20", "EMA_20", "RSI_14",
        "BB_Middle", "BB_Upper", "BB_Lower", "EMA_12", "EMA_26",
        "MACD", "%K", "%D", "Momentum", "WMA_20",
        "TRIX", "VWAP", "CCI", "Bias", "RVI", "Williams %R", "MFI_14",
    ]
    known_reals = ["time_idx"] + additional_features

    # 5. 划分训练集和验证集
    training_data = data[data.time_idx <= training_cutoff]
    validation_data = data[data.time_idx > training_cutoff]
    print(f"验证集数量: {len(validation_data)}")

    training_dataset = TimeSeriesDataSet(
        data=training_data,
        time_idx="time_idx",
        target="Close",
        group_ids=["stock_id"],
        categorical_encoders={"stock_id": NaNLabelEncoder().fit(data["stock_id"])},
        static_categoricals=["static"],
        time_varying_known_reals=known_reals,
        time_varying_unknown_reals=["Close"],
        min_encoder_length=max_encoder_length,
        max_encoder_length=max_encoder_length,
        min_prediction_length=max_prediction_length,
        max_prediction_length=max_prediction_length,
        target_normalizer=GroupNormalizer(groups=["stock_id"], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        randomize_length=None,
    )

    validation_dataset = TimeSeriesDataSet.from_dataset(
        training_dataset,
        validation_data,
        stop_randomization=True,
    )

    batch_size = 64
    train_dataloader = training_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=2)
    val_dataloader = validation_dataset.to_dataloader(train=False, batch_size=batch_size, num_workers=2)

    early_stop_callback = EarlyStopping(
        monitor="val_loss", min_delta=1e-4, patience=5, verbose=True, mode="min"
    )
    lr_logger = LearningRateMonitor()

    trainer = pl.Trainer(
        max_epochs=200,
        accelerator="gpu",   # 如果没有 GPU，可设为 "cpu"
        devices="auto",
        gradient_clip_val=0.1,
        limit_train_batches=30,
        limit_val_batches=3,
        callbacks=[lr_logger, early_stop_callback],
    )

    # 构造 TFT 模型，设置 output_size 为 1，仅输出中位数预测
    tft = TemporalFusionTransformer.from_dataset(
        training_dataset,
        learning_rate=0.03,
        hidden_size=16,
        attention_head_size=1,
        dropout=0.1,
        hidden_continuous_size=8,
        output_size=1,
        loss=MAE(),
        log_interval=10,
        reduce_on_plateau_patience=4,
    )

    print(f"模型参数数量: {tft.size() / 1e3:.1f}k")

    torch.set_num_threads(10)

    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    # 预测时确保两边的张量在同一设备上
    predictions = tft.predict(val_dataloader, mode="prediction")
    actuals = torch.cat([y for x, (y, _) in iter(val_dataloader)])
    actuals = actuals.to(predictions.device)  # 将实际值移动到 predictions 的设备上
    mae = (actuals - predictions).abs().mean()
    print(f"模型平均绝对误差 (MAE): {mae.item():.4f}")

if __name__ == '__main__':
    main()

INFO: Seed set to 42
/tmp/ipykernel_125/1226449937.py:33: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


剔除异常列: ['#NAME?', '#NAME?.1']
数据总长度: 1437, 训练截止点: 1406
验证集数量: 31


/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | MAE                             | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings              

模型参数数量: 48.7k


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Monitored metric val_loss did not improve in the last 5 records. Best score: 0.182. Signaling Trainer to stop.
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


模型平均绝对误差 (MAE): 1.7601


In [38]:
import warnings
import pandas as pd
import torch
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_forecasting import TimeSeriesDataSet, GroupNormalizer, TemporalFusionTransformer, QuantileLoss
from pytorch_forecasting.data import NaNLabelEncoder

# 忽略 pandas 的 SettingWithCopyWarning
warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)

def main():
    # 设置随机种子，确保结果可复现
    pl.seed_everything(42, workers=True)

    # 1. 加载数据（使用 Kaggle 上的文件路径）
    file_path = "/kaggle/input/vhzmdd/AAPL_stock_data_with_indicators.csv"
    data = pd.read_csv(file_path)

    # 2. 数据预处理
    # 转换日期格式，注意日期格式需和 CSV 文件一致
    data["Date"] = pd.to_datetime(data["Date"], format="%Y/%m/%d")
    data = data.sort_values("Date").reset_index(drop=True)
    data["time_idx"] = data.index + 1  # 时间索引从 1 开始
    # 假设数据只包含单只股票，将 stock_id 固定为 "AAPL"
    data["stock_id"] = "AAPL"
    data["static"] = data["stock_id"]

    # 剔除列名中包含异常字符串（如 "#NAME?"）的列
    drop_cols = [col for col in data.columns if "#NAME?" in col]
    if drop_cols:
        print(f"剔除异常列: {drop_cols}")
        data = data.drop(columns=drop_cols)

    # 前向填充缺失值
    data = data.fillna(method='ffill')

    # 3. 定义采样参数
    max_encoder_length = 30
    max_prediction_length = 2
    training_cutoff = data["time_idx"].max() - (max_encoder_length + max_prediction_length)
    print(f"数据总长度: {data['time_idx'].max()}, 训练截止点: {training_cutoff}")

    # 4. 定义额外技术指标（确保与 CSV 列名完全一致）
    additional_features = [
        "Open", "High", "Low", "Volume", "SMA_20", "EMA_20", "RSI_14",
        "BB_Middle", "BB_Upper", "BB_Lower", "EMA_12", "EMA_26",
        "MACD", "MACD_Signal", "ATR_14", "%K", "%D", "Momentum", "WMA_20",
        "TR", "DM+", "DM-", "DX", "ADX_14", "SMA_Smooth_20", "Volatility",
        "Upper_Price_Channel", "Lower_Price_Channel", "Keltner_Middle",
        "Keltner_Upper", "Keltner_Lower"
    ]
    known_reals = ["time_idx"] + additional_features

    # 5. 构造训练集与验证集
    training_data = data[data.time_idx <= training_cutoff]
    validation_data = data[data.time_idx > training_cutoff]
    print(f"验证集数量: {len(validation_data)}")

    training_dataset = TimeSeriesDataSet(
        data=training_data,
        time_idx="time_idx",
        target="Close",
        group_ids=["stock_id"],
        categorical_encoders={"stock_id": NaNLabelEncoder().fit(data["stock_id"])},
        static_categoricals=["static"],
        time_varying_known_reals=known_reals,         # 使用额外的技术指标
        time_varying_unknown_reals=["Close"],           # 目标为 Close
        min_encoder_length=max_encoder_length,
        max_encoder_length=max_encoder_length,
        min_prediction_length=max_prediction_length,
        max_prediction_length=max_prediction_length,
        target_normalizer=GroupNormalizer(groups=["stock_id"], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        randomize_length=None,
    )

    validation_dataset = TimeSeriesDataSet.from_dataset(
        training_dataset,
        validation_data,
        stop_randomization=True,
    )

    # 6. 构造 DataLoader
    batch_size = 64
    train_dataloader = training_dataset.to_dataloader(
        train=True, batch_size=batch_size, num_workers=2, persistent_workers=True
    )
    val_dataloader = validation_dataset.to_dataloader(
        train=False, batch_size=batch_size, num_workers=2, persistent_workers=True
    )

    # 7. 定义回调函数和日志记录
    early_stop_callback = EarlyStopping(
        monitor="val_loss", min_delta=1e-4, patience=1, verbose=True, mode="min"
    )
    lr_logger = LearningRateMonitor()
    tensorboard_logger = TensorBoardLogger("lightning_logs")

    trainer = pl.Trainer(
        max_epochs=200,
        accelerator="gpu",   # 使用 GPU，如无 GPU 则设置为 "cpu"
        devices=1,
        gradient_clip_val=0.1,
        limit_train_batches=30,  # 调试时每个 epoch 限制 30 个批次
        callbacks=[lr_logger, early_stop_callback],
        logger=tensorboard_logger
    )

    # 8. 构造 TemporalFusionTransformer 模型
    # 此处使用 QuantileLoss，默认设置量化分位数为 [0.1, 0.5, 0.9]
    # 可根据需要调整 hidden_size、dropout、hidden_continuous_size 等参数以提高预测精度
    tft = TemporalFusionTransformer.from_dataset(
        training_dataset,
        hidden_size=32,
        attention_head_size=1,
        dropout=0.1,
        hidden_continuous_size=16,
        loss=QuantileLoss(),
        log_interval=2,
        learning_rate=0.03,
        reduce_on_plateau_patience=4
    )
    print(f"模型参数数量: {tft.size() / 1e3:.1f}k")

    torch.set_num_threads(10)

    # 9. 开始训练
    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    # 10. 验证集预测与误差计算
    # 使用 "quantiles" 模式获得预测，注意不要传入多余的 quantiles 关键字参数
    predictions = tft.predict(val_dataloader, mode="quantiles")
    # 默认 QuantileLoss 设置了三个 quantiles：[0.1, 0.5, 0.9]，因此预测输出 shape 为 [batch, prediction_length, 3]
    # 选取中位数（quantile=0.5）对应的预测，即取索引 1 的值
    median_predictions = predictions[..., 1]
    # 拼接批次中的实际目标值，并确保在同一设备上
    actuals = torch.cat([y for x, (y, _) in iter(val_dataloader)])
    actuals = actuals.to(median_predictions.device)
    mae = (actuals - median_predictions).abs().mean()
    print(f"模型平均绝对误差 (MAE): {mae.item():.4f}")

if __name__ == '__main__':
    main()

INFO: Seed set to 42
/tmp/ipykernel_125/2146841726.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


剔除异常列: ['#NAME?', '#NAME?.1']
数据总长度: 1437, 训练截止点: 1405
验证集数量: 32


INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      

模型参数数量: 222.9k


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Metric val_loss improved by 4.113 >= min_delta = 0.0001. New best score: 2.580


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Monitored metric val_loss did not improve in the last 1 records. Best score: 2.580. Signaling Trainer to stop.
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


模型平均绝对误差 (MAE): 15.0787


In [43]:
import warnings
import pandas as pd
import torch
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, GroupNormalizer
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.models.nn.rnn import LSTM as ForecastingLSTM

# 忽略 pandas 的 SettingWithCopyWarning
warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)


class LSTMModel(pl.LightningModule):
    def __init__(self, input_size, hidden_size, prediction_length, num_layers=1, lr=1e-3):
        """
        :param input_size: 输入特征数
        :param hidden_size: LSTM 隐藏层维度
        :param prediction_length: 预测区间长度
        :param num_layers: LSTM 层数
        :param lr: 学习率
        """
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr
        # 使用 pytorch_forecasting 中的 LSTM 层（设置 batch_first=True）
        self.lstm = ForecastingLSTM(
            input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True
        )
        # 使用全连接层，将最后的隐藏状态映射到预测区间
        self.fc = torch.nn.Linear(hidden_size, prediction_length)
        self.loss_fn = torch.nn.MSELoss()

    def forward(self, x):
        """
        前向传播：使用 x["encoder_cont"] (形状为 (batch, encoder_length, num_features))
        """
        encoder_input = x["encoder_cont"]
        # lstm_out: (batch, encoder_length, hidden_size)
        # h_n: (num_layers, batch, hidden_size)
        _, (h_n, _) = self.lstm(encoder_input)
        # 使用最后一层隐藏状态 (batch, hidden_size)
        last_hidden = h_n[-1]
        prediction = self.fc(last_hidden)  # (batch, prediction_length)
        prediction = prediction.unsqueeze(-1)  # (batch, prediction_length, 1)
        return prediction

    def training_step(self, batch, batch_idx):
        x, (y, weight) = batch
        y_hat = self(x)
        # 调整目标维度为 (batch, prediction_length, 1)
        # 若 y 的形状不符合要求，则进行变换
        if y.dim() == 2:
            y = y.unsqueeze(-1)
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, (y, weight) = batch
        y_hat = self(x)
        if y.dim() == 2:
            y = y.unsqueeze(-1)
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode="min", patience=3, factor=0.5, verbose=True
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss',
                'frequency': 1,
            }
        }


def main():
    pl.seed_everything(42, workers=True)

    # 1. 加载数据（使用 Kaggle 上的文件路径，新数据需包含丰富的技术指标）
    file_path = "/kaggle/input/vhzmdd/AAPL_stock_data_with_indicators.csv"
    data = pd.read_csv(file_path)

    # 2. 数据预处理
    # 将日期转换为 datetime 类型，并按照日期排序
    data["Date"] = pd.to_datetime(data["Date"], format="%Y/%m/%d")
    data = data.sort_values("Date").reset_index(drop=True)
    data["time_idx"] = data.index + 1  # 构造时间索引，从 1 开始
    # 假设数据只包含单只股票，将 stock_id 固定为 "AAPL"
    data["stock_id"] = "AAPL"
    data["static"] = data["stock_id"]

    # 剔除列名中包含异常字符串（如 "#NAME?"）的列
    drop_cols = [col for col in data.columns if "#NAME?" in col]
    if drop_cols:
        print(f"剔除异常列: {drop_cols}")
        data = data.drop(columns=drop_cols)

    # 前向填充缺失值
    data = data.fillna(method='ffill')

    # 3. 设置采样参数
    max_encoder_length = 60
    max_prediction_length = 1
    training_cutoff = data["time_idx"].max() - (max_encoder_length + max_prediction_length)
    print(f"数据总长度: {data['time_idx'].max()}, 训练截止点: {training_cutoff}")

    # 4. 定义额外技术指标（请确保 CSV 文件中包含对应的列）
    additional_features = [
        "Open", "High", "Low", "Volume", "SMA_20", "EMA_20", "RSI_14",
        "BB_Middle", "BB_Upper", "BB_Lower", "EMA_12", "EMA_26",
        "MACD", "MACD_Signal", "ATR_14", "%K", "%D", "Momentum", "WMA_20",
    ]
    # 将时间索引与额外指标合并
    known_reals = ["time_idx"] + additional_features

    # 5. 构造训练集与验证集数据集
    training_data = data[data.time_idx <= training_cutoff]
    validation_data = data[data.time_idx > training_cutoff]

    print(f"验证集数量: {len(validation_data)}")

    training_dataset = TimeSeriesDataSet(
        data=training_data,
        time_idx="time_idx",
        target="Close",
        group_ids=["stock_id"],
        categorical_encoders={"stock_id": NaNLabelEncoder().fit(data["stock_id"])},
        static_categoricals=["static"],
        time_varying_known_reals=known_reals,         # 使用额外技术指标作为已知特征
        time_varying_unknown_reals=["Close"],           # 目标为 Close
        min_encoder_length=max_encoder_length,
        max_encoder_length=max_encoder_length,
        min_prediction_length=max_prediction_length,
        max_prediction_length=max_prediction_length,
        target_normalizer=GroupNormalizer(groups=["stock_id"], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        randomize_length=None,
    )

    validation_dataset = TimeSeriesDataSet.from_dataset(
        training_dataset,
        validation_data,
        stop_randomization=True,
    )

    # 6. 构造 DataLoader
    batch_size = 64
    train_dataloader = training_dataset.to_dataloader(
        train=True, batch_size=batch_size, num_workers=2, persistent_workers=True
    )
    val_dataloader = validation_dataset.to_dataloader(
        train=False, batch_size=batch_size, num_workers=2, persistent_workers=True
    )

    # 7. 定义回调函数与日志记录
    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5, verbose=True, mode="min")
    lr_logger = LearningRateMonitor(logging_interval="epoch")
    
    trainer = pl.Trainer(
        max_epochs=300,
        accelerator="gpu",  # 如无 GPU，请设置为 "cpu"
        devices="auto",
        gradient_clip_val=0.1,
        callbacks=[lr_logger, early_stop_callback],
    )

    # 打印输入特征数
    sample_batch = next(iter(train_dataloader))
    input_size = sample_batch[0]["encoder_cont"].shape[-1]
    print(f"Input feature size for LSTM: {input_size}")

    # 8. 构造 LSTM 模型
    lstm_model = LSTMModel(
        input_size=input_size,
        hidden_size=128,
        prediction_length=max_prediction_length,
        num_layers=1,
        lr=1e-3
    )
    total_params = sum(p.numel() for p in lstm_model.parameters())
    print(f"Number of parameters in network: {total_params/1e3:.1f}k")

    torch.set_num_threads(10)
    trainer.fit(lstm_model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

    # 9. 评估阶段：注意预测和实际值均为归一化后的结果
    lstm_model.eval()
    predictions_list = []
    actuals_list = []
    with torch.no_grad():
        for batch in val_dataloader:
            x, (y, weight) = batch
            y_hat = lstm_model(x)
            predictions_list.append(y_hat)
            actuals_list.append(y)
    predictions = torch.cat(predictions_list, dim=0)
    actuals = torch.cat(actuals_list, dim=0)
    mae = (actuals - predictions).abs().mean()
    print(f"Mean absolute error: {mae.item():.4f}")

if __name__ == '__main__':
    main()

INFO: Seed set to 42
/tmp/ipykernel_125/3002250319.py:106: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


剔除异常列: ['#NAME?', '#NAME?.1']
数据总长度: 1437, 训练截止点: 1376
验证集数量: 61


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | lstm    | LSTM    | 78.8 K | train
1 | fc      | Linear  | 129    | train
2 | loss_fn | MSELoss | 0      | train
--------------------------------------------
79.0 K    Trainable params
0         Non-trainable params
79.0 K    Total params
0.316     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Input feature size for LSTM: 24
Number of parameters in network: 79.0k


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Monitored metric val_loss did not improve in the last 5 records. Best score: 57.051. Signaling Trainer to stop.


Mean absolute error: 7.9381


In [46]:
import warnings
import pandas as pd
import torch
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, GroupNormalizer, NHiTS, MQF2DistributionLoss
from pytorch_forecasting.data import NaNLabelEncoder

# 忽略 pandas 的 SettingWithCopyWarning
warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)

def main():
    # 固定随机种子，保证结果可复现
    pl.seed_everything(42, workers=True)

    # 1. 加载数据
    # 如果你使用 Kaggle 数据，请修改为相应路径
    file_path = "/kaggle/input/vhzmdd/AAPL_stock_data_with_indicators.csv"
    data = pd.read_csv(file_path)

    # 2. 数据预处理
    data["Date"] = pd.to_datetime(data["Date"], format="%Y/%m/%d")
    data = data.sort_values("Date").reset_index(drop=True)
    data["time_idx"] = data.index + 1  # 时间索引从 1 开始
    # 假设数据仅包含单只股票，将 stock_id 固定
    data["stock_id"] = "AAPL"
    data["static"] = data["stock_id"]

    # 剔除含异常字符串（如 "#NAME?"）的列
    drop_cols = [col for col in data.columns if "#NAME?" in col]
    if drop_cols:
        print(f"剔除异常列: {drop_cols}")
        data = data.drop(columns=drop_cols)

    # 前向填充缺失值
    data = data.fillna(method='ffill')

    # 3. 设置采样参数
    max_encoder_length = 30
    max_prediction_length = 10
    training_cutoff = data["time_idx"].max() - (max_encoder_length + max_prediction_length)
    print(f"数据总长度: {data['time_idx'].max()}, 训练截止点: {training_cutoff}")

    # 4. 定义额外技术指标（请确保数据中包含对应的列，如 Open, High, Low 等）
    additional_features = [
        "Open", "High", "Low", "Volume", "SMA_20", "EMA_20", "RSI_14",
        "BB_Middle", "BB_Upper", "BB_Lower", "EMA_12", "EMA_26",
        "MACD", "MACD_Signal", "ATR_14", "%K", "%D", "Momentum", "WMA_20"
    ]
    known_reals = ["time_idx"] + additional_features

    # 5. 构造训练集
    training_data = data[data.time_idx <= training_cutoff]
    training_dataset = TimeSeriesDataSet(
        data=training_data,
        time_idx="time_idx",
        target="Close",
        group_ids=["stock_id"],
        categorical_encoders={"stock_id": NaNLabelEncoder().fit(data["stock_id"])},
        static_categoricals=["static"],
        min_encoder_length=max_encoder_length,
        max_encoder_length=max_encoder_length,
        min_prediction_length=max_prediction_length,
        max_prediction_length=max_prediction_length,
        # 目标仅为 Close
        time_varying_unknown_reals=["Close"],
        # 已知变量包含时间索引和额外技术指标
        time_varying_known_reals=known_reals,
        target_normalizer=GroupNormalizer(groups=["stock_id"], transformation="softplus"),
        # 对 NHiTS 此处设置为 False，若需要可考虑设置为 True
        add_relative_time_idx=False,
        add_target_scales=True,
        randomize_length=None,
    )

    # 构造验证集，起始位置设置为训练截止点之后
    validation_dataset = TimeSeriesDataSet.from_dataset(
        training_dataset, data, min_prediction_idx=training_cutoff + 1
    )

    batch_size = 64
    train_dataloader = training_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=8)
    val_dataloader = validation_dataset.to_dataloader(train=False, batch_size=batch_size, num_workers=8)

    # 6. 设置回调：EarlyStopping与LearningRateMonitor
    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor(logging_interval="epoch")

    trainer = pl.Trainer(
        max_epochs=10,
        accelerator="gpu",  # 如无 GPU，可设置为 "cpu"
        enable_model_summary=True,
        gradient_clip_val=1.0,
        callbacks=[lr_logger, early_stop_callback],
        limit_train_batches=30,
        enable_checkpointing=True,
    )

    # 7. 构造 NHiTS 模型
    net = NHiTS.from_dataset(
        training_dataset,
        learning_rate=5e-3,
        log_interval=10,
        log_val_interval=1,
        weight_decay=1e-2,
        backcast_loss_ratio=0.0,
        hidden_size=64,
        optimizer="AdamW",
        loss=MQF2DistributionLoss(prediction_length=max_prediction_length),
    )

    print(f"模型参数数量: {net.size() / 1e3:.1f}k")
    torch.set_num_threads(10)
    
    # 8. 开始训练
    trainer.fit(
        net,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    # 9. 验证集预测与误差计算（此处以平均绝对误差 MAE 为例，注意数据为归一化后的结果）
    actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
    predictions = net.predict(val_dataloader)
    device = predictions.device
    actuals = actuals.to(device)
    mae = (actuals - predictions).abs().mean()
    print(f"Mean absolute error of model: {mae.item():.4f}")

if __name__ == '__main__':
    main()

INFO: Seed set to 42
/tmp/ipykernel_125/4203790303.py:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


剔除异常列: ['#NAME?', '#NAME?.1']
数据总长度: 1437, 训练截止点: 1397


/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name            | Type                 | Params | Mode 
-----------------------------------------------------------------
0 | loss            | MQF2DistributionLoss | 3.5 K  | train
1 | logging_metrics | ModuleList           | 0      | train
2 | embeddings      | MultiEmbedding       | 1      | train
3 | model           | NHiTS                | 192 K  | train
---------------------------------------

模型参数数量: 196.2k


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Mean absolute error of model: 3.1569


In [45]:
!pip install cpflows

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for cpflows: filename=cpflows-0.1.2-py3-none-any.whl size=54295 sha256=b10cd662e761a9011a62e084c8e8c52e2b649128c2fed3a33e0618a19fa9bae8
  Stored in directory: /root/.cache/pip/wheels/49/00/ad/0b1b6c1e1ea1b85f7aa4eaefb0cf3890c2fd2b5c5465f10757
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6489 sha256=87f9196a56900623989a644771bf529e2dbf9aeaf4ac1c1dc13547f1f834a9cb
  Stored in directory: /root/.cache/pip/wheels/38/b6/54/4477e68f0e313dd4c43b79e6ed43bdcf12abe7626d6d81b369
Successfully built cpflows subprocess32
